# Fair GAN Implementation for Adult Census Dataset

## Setup and Installation

In [3]:
%pip install ucimlrepo
%pip install tensorflow>=2.4.0
%pip install aif360


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
zsh:1: 2.4.0 not found
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 6.7 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from sklearn.preprocessing import StandardScaler, LabelEncoder
from ucimlrepo import fetch_ucirepo
import ssl
import warnings

ssl._create_default_https_context = ssl._create_unverified_context
warnings.filterwarnings('ignore')

print("TensorFlow version:", tf.__version__)

pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


TensorFlow version: 2.15.0


In [202]:
# Fetch and prepare dataset
adult = fetch_ucirepo(id=2)
X = adult.data.features
y = adult.data.targets
y.columns = ['income']
full_data = pd.concat([X, y], axis=1)

print("Dataset shape:", full_data.shape)
print("\nColumns:", full_data.columns.tolist())

Dataset shape: (48842, 15)

Columns: ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']


In [203]:
def preprocess_data(df):
    df_copy = df.copy()
    
    categorical_features = ['workclass', 'education', 'marital-status', 'occupation',
                          'relationship', 'race', 'sex', 'native-country']
    
    numerical_features = ['age', 'fnlwgt', 'education-num', 'capital-gain',
                         'capital-loss', 'hours-per-week']
    
    label_encoders = {}
    for feature in categorical_features:
        label_encoders[feature] = LabelEncoder()
        df_copy[feature] = label_encoders[feature].fit_transform(df_copy[feature].astype(str))
    
    df_copy['income'] = df_copy['income'].apply(lambda x: 1 if x == '>50K' else 0)
    
    scaler = StandardScaler()
    df_copy[numerical_features] = scaler.fit_transform(df_copy[numerical_features])
    
    return df_copy, label_encoders, scaler

processed_data, label_encoders, scaler = preprocess_data(full_data)

In [204]:
class GAN_with_FairnessDebiasing(tf.keras.Model):
    def __init__(self, input_dim, latent_dim=100):
        super(GAN_with_FairnessDebiasing, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.last_d_loss = 1.0
        
        # Generator with conditioning
        # size of the input can be adjusted 
        self.generator = tf.keras.Sequential([
            tf.keras.layers.Dense(256, input_dim=latent_dim + 2),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            
            tf.keras.layers.Dense(512),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            
            tf.keras.layers.Dense(input_dim),
            tf.keras.layers.Activation('sigmoid')
        ])
        
        # Main discriminator
        self.discriminator = tf.keras.Sequential([
            tf.keras.layers.Dense(128, input_dim=input_dim),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.Dropout(0.3),
            
            tf.keras.layers.Dense(64),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.Dropout(0.3),
            
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        
        # Fairness discriminator
        self.fairness_discriminator = tf.keras.Sequential([
            tf.keras.layers.Dense(128, input_dim=input_dim),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(64),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.Dense(2, activation='sigmoid')  # Changed to output 2 values for sex and race
        ])
        
        # Learning rate can be adjusted
        self.g_optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001, beta_1=0.5)
        self.d_optimizer = tf.keras.optimizers.Adam(learning_rate=00.00001, beta_1=0.5)
        self.f_optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001, beta_1=0.5)
        self.loss_fn = tf.keras.losses.BinaryCrossentropy()

    def demographic_parity_loss(self, data):
        # Extract protected features
        sex = data[:, 9:10]  
        race = data[:, 8:9]  
        protected_features = tf.concat([sex, race], axis=1)
        
        predictions = self.discriminator(data)
        
        # Calculate means for different demographic groups
        priv_mask = tf.reduce_all(tf.greater(protected_features, 0.5), axis=1)
        unpriv_mask = tf.reduce_all(tf.less_equal(protected_features, 0.5), axis=1)
        
        # Ensure mask has correct shape
        priv_mask = tf.reshape(priv_mask, [-1])
        unpriv_mask = tf.reshape(unpriv_mask, [-1])
        
        # Safe handling of empty groups
        priv_preds = tf.boolean_mask(predictions, priv_mask)
        unpriv_preds = tf.boolean_mask(predictions, unpriv_mask)
        
        priv_mean = tf.reduce_mean(priv_preds) if tf.size(priv_preds) > 0 else 0.0
        unpriv_mean = tf.reduce_mean(unpriv_preds) if tf.size(unpriv_preds) > 0 else 0.0
        
        return tf.abs(priv_mean - unpriv_mean)

    def equalized_odds_loss(self, data, labels):
        # Extract protected features
        sex = data[:, 9:10]  # sex column
        race = data[:, 8:9]  # race column
        protected_features = tf.concat([sex, race], axis=1)
        
        predictions = self.discriminator(data)
        
        # Ensure labels have correct shape
        labels = tf.cast(tf.reshape(labels, [-1, 1]), tf.float32)
        
        # Calculate true positive rates for different groups
        priv_mask = tf.reduce_all(tf.greater(protected_features, 0.5), axis=1)
        unpriv_mask = tf.reduce_all(tf.less_equal(protected_features, 0.5), axis=1)
        positive_mask = tf.reshape(tf.greater(labels, 0.5), [-1])
        
        # Safe handling of group intersections
        priv_pos_mask = tf.logical_and(priv_mask, positive_mask)
        unpriv_pos_mask = tf.logical_and(unpriv_mask, positive_mask)
        
        priv_pos_preds = tf.boolean_mask(predictions, priv_pos_mask)
        unpriv_pos_preds = tf.boolean_mask(predictions, unpriv_pos_mask)
        
        priv_tpr = tf.reduce_mean(priv_pos_preds) if tf.size(priv_pos_preds) > 0 else 0.0
        unpriv_tpr = tf.reduce_mean(unpriv_pos_preds) if tf.size(unpriv_pos_preds) > 0 else 0.0
        
        return tf.abs(priv_tpr - unpriv_tpr)

    def train_step(self, real_data):
        batch_size = tf.shape(real_data)[0]
        
        # Extract labels and protected attributes
        labels = real_data[:, -1]  # income is the last column
        protected_features = tf.concat([
            real_data[:, 9:10],
            real_data[:, 8:9]    
        ], axis=1)
        
        # Generate balanced conditions
        sex = tf.random.uniform([batch_size, 1], 0.4, 0.6) > 0.5
        race = tf.random.uniform([batch_size, 1], 0.4, 0.6) > 0.5
        conditions = tf.concat([tf.cast(sex, tf.float32), 
                              tf.cast(race, tf.float32)], axis=1)
        
        # Train Generator
        noise = tf.random.normal([batch_size, self.latent_dim])
        conditional_noise = tf.concat([noise, conditions], axis=1)
        
        with tf.GradientTape() as gen_tape:
            generated_data = self.generator(conditional_noise, training=True)
            fake_output = self.discriminator(generated_data, training=True)
            
            gen_loss = self.loss_fn(tf.ones_like(fake_output), fake_output)
            dp_loss = self.demographic_parity_loss(generated_data)
            eo_loss = self.equalized_odds_loss(generated_data, labels)
            
            fairness_weight = 0.5
            total_gen_loss = gen_loss + fairness_weight * (dp_loss + eo_loss)
        
        gen_grads = gen_tape.gradient(total_gen_loss, self.generator.trainable_variables)
        self.g_optimizer.apply_gradients(zip(gen_grads, self.generator.trainable_variables))
        
        # Train Discriminator
        with tf.GradientTape() as disc_tape:
            generated_data = self.generator(conditional_noise, training=True)
            real_output = self.discriminator(real_data, training=True)
            fake_output = self.discriminator(generated_data, training=True)
            
            # These can be adjusted 
            real_labels = tf.random.uniform([batch_size, 1], 0.8, 0.9)
            fake_labels = tf.random.uniform([batch_size, 1], 0.1, 0.2)
            
            real_loss = self.loss_fn(real_labels, real_output)
            fake_loss = self.loss_fn(fake_labels, fake_output)
            disc_loss = (real_loss + fake_loss) / 2
            
            dp_reg = self.demographic_parity_loss(real_data)
            eo_reg = self.equalized_odds_loss(real_data, labels)
            total_disc_loss = disc_loss + fairness_weight * (dp_reg + eo_reg)
        
        if disc_loss < 1.0:
            disc_grads = disc_tape.gradient(total_disc_loss, self.discriminator.trainable_variables)
            self.d_optimizer.apply_gradients(zip(disc_grads, self.discriminator.trainable_variables))
        
        # Train Fairness Discriminator
        with tf.GradientTape() as fair_tape:
            fair_pred_real = self.fairness_discriminator(real_data, training=True)
            fair_pred_fake = self.fairness_discriminator(generated_data, training=True)
            
            fair_loss = (self.loss_fn(protected_features, fair_pred_real) + 
                        self.loss_fn(conditions, fair_pred_fake)) / 2
        
        fair_grads = fair_tape.gradient(fair_loss, self.fairness_discriminator.trainable_variables)
        self.f_optimizer.apply_gradients(zip(fair_grads, self.fairness_discriminator.trainable_variables))
        
        return float(total_disc_loss.numpy()), float(total_gen_loss.numpy())

    def train(self, dataset, epochs=10, batch_size=512):
        dataset = tf.data.Dataset.from_tensor_slices(dataset).shuffle(1000).batch(batch_size)
        
        for epoch in range(epochs):
            total_d_loss = 0.0
            total_g_loss = 0.0
            num_batches = 0
            
            for batch in dataset:
                d_loss, g_loss = self.train_step(batch)
                total_d_loss += d_loss
                total_g_loss += g_loss
                num_batches += 1
            
            avg_d_loss = total_d_loss / num_batches
            avg_g_loss = total_g_loss / num_batches
            
            print(f"Epoch {epoch+1}/{epochs}, D Loss: {avg_d_loss:.4f}, G Loss: {avg_g_loss:.4f}")
            
    def generate_samples(self, num_samples):
        # Generate balanced conditions
        sex = np.random.binomial(1, 0.5, (num_samples, 1))
        race = np.random.binomial(1, 0.5, (num_samples, 1))
        conditions = np.concatenate([sex, race], axis=1)
        
        # Generate samples with conditions
        noise = tf.random.normal([num_samples, self.latent_dim])
        conditional_noise = tf.concat([noise, tf.convert_to_tensor(conditions, dtype=tf.float32)], axis=1)
        generated = self.generator(conditional_noise).numpy()
        
        # Create DataFrame
        generated_df = pd.DataFrame(generated, columns=processed_data.columns)
        
        # Ensure binary values
        binary_features = ['income', 'sex', 'race']
        for feature in binary_features:
            generated_df[feature] = (generated_df[feature] > 0.5).astype(float)
        
        return generated_df

In [212]:
# Initialize and train FairGAN
gan = GAN_with_FairnessDebiasing(input_dim=processed_data.shape[1])
print("Starting training...")
gan.train(processed_data.values, epochs=20, batch_size=10000)

# Generate synthetic data
synthetic_samples = gan.generate_samples(1000)
synthetic_df = pd.DataFrame(synthetic_samples, columns=processed_data.columns)
print (synthetic_df.head())


Starting training...
Epoch 1/20, D Loss: 1.3118, G Loss: 0.6600
Epoch 2/20, D Loss: 1.3457, G Loss: 0.6582
Epoch 3/20, D Loss: 1.3000, G Loss: 0.6564
Epoch 4/20, D Loss: 1.3535, G Loss: 0.6547
Epoch 5/20, D Loss: 1.3533, G Loss: 0.6534
Epoch 6/20, D Loss: 1.3596, G Loss: 0.6522
Epoch 7/20, D Loss: 1.3039, G Loss: 0.6506
Epoch 8/20, D Loss: 1.4044, G Loss: 0.6491
Epoch 9/20, D Loss: 1.3593, G Loss: 0.6479
Epoch 10/20, D Loss: 1.3089, G Loss: 0.6465
Epoch 11/20, D Loss: 1.3661, G Loss: 0.6460
Epoch 12/20, D Loss: 1.3144, G Loss: 0.6444
Epoch 13/20, D Loss: 1.3632, G Loss: 0.6444
Epoch 14/20, D Loss: 1.3089, G Loss: 0.6435
Epoch 15/20, D Loss: 1.3519, G Loss: 0.6427
Epoch 16/20, D Loss: 1.3577, G Loss: 0.6416
Epoch 17/20, D Loss: 1.3597, G Loss: 0.6419
Epoch 18/20, D Loss: 1.3089, G Loss: 0.6397
Epoch 19/20, D Loss: 1.3560, G Loss: 0.6389
Epoch 20/20, D Loss: 1.3607, G Loss: 0.6386
        age  workclass    fnlwgt  education  education-num  marital-status  \
0  0.508776   0.526309  0.7340

In [213]:
def evaluate_fairness(dataset, protected_attribute):
    metrics = BinaryLabelDatasetMetric(
        dataset, 
        unprivileged_groups=[{protected_attribute: 0}],
        privileged_groups=[{protected_attribute: 1}]
    )
    
    print(f"\nFairness Metrics for {protected_attribute}:")
    print(f"Disparate Impact: {metrics.disparate_impact():.3f}")
    print(f"Statistical Parity Difference: {metrics.statistical_parity_difference():.3f}")
# Create AIF360 dataset with label specifications
aif_dataset = BinaryLabelDataset(
    df=processed_data,
    label_names=['income'],
    protected_attribute_names=['sex', 'race'],
    privileged_protected_attributes=[1, 1],
    favorable_label=1,
    unfavorable_label=0
)

# Print original metrics
print("\nOriginal Data Metrics:")
evaluate_fairness(aif_dataset, 'sex')
evaluate_fairness(aif_dataset, 'race')

# Generate synthetic data
synthetic_samples = gan.generate_samples(1000)
synthetic_df = pd.DataFrame(synthetic_samples, columns=processed_data.columns)

# Round the synthetic data to ensure binary values
synthetic_df['income'] = synthetic_df['income'].round().clip(0, 1)  # Ensure values are 0 or 1
synthetic_df['sex'] = synthetic_df['sex'].round().clip(0, 1)       # Ensure values are 0 or 1
synthetic_df['race'] = synthetic_df['race'].round().clip(0, 1)     # Ensure values are 0 or 1

# Create synthetic dataset
synthetic_aif_dataset = BinaryLabelDataset(
    df=synthetic_df,
    label_names=['income'],
    protected_attribute_names=['sex', 'race'],
    privileged_protected_attributes=[1, 1],
    favorable_label=1,
    unfavorable_label=0
)

print("\nSynthetic Data Metrics:")
evaluate_fairness(synthetic_aif_dataset, 'sex')
evaluate_fairness(synthetic_aif_dataset, 'race')

# Optional: Print distribution of values to verify
print("\nSynthetic Data Distribution:")
print("Income distribution:", synthetic_df['income'].value_counts())
print("Sex distribution:", synthetic_df['sex'].value_counts())
print("Race distribution:", synthetic_df['race'].value_counts())


Original Data Metrics:

Fairness Metrics for sex:
Disparate Impact: 0.357
Statistical Parity Difference: -0.131

Fairness Metrics for race:
Disparate Impact: 0.422
Statistical Parity Difference: -0.105

Synthetic Data Metrics:

Fairness Metrics for sex:
Disparate Impact: 0.675
Statistical Parity Difference: -0.118

Fairness Metrics for race:
Disparate Impact: 1.139
Statistical Parity Difference: 0.039

Synthetic Data Distribution:
Income distribution: income
0.0    702
1.0    298
Name: count, dtype: int64
Sex distribution: sex
0.0    561
1.0    439
Name: count, dtype: int64
Race distribution: race
0.0    518
1.0    482
Name: count, dtype: int64
